In [327]:
#this will be the basic predictor that others will be compared to 
#uploading libraries 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import string

In [328]:
#preparing the data 
X = pd.read_csv("Combined Data.csv", header=0, index_col=0)
X = X.dropna()
y = X['status']
X = X.drop(columns='status')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=43)
print(X_train.iloc[0])



statement    Am I depressed because I don't wanna escape fr...
Name: 47395, dtype: object


In [329]:
X_train.columns
X_train.head()
X_test

,statement
40095,i m a m recently diagnosed with autism and adh...
42690,ripped switcheasy color
30592,would you like to see a movie with me and my f...
6960,Will I be able to feel pleasure in worshiping ...
41262,andygaming une seule issue changer il n y aura...
...,...
8890,I do not want to exist anymore. Traveling and ...
18980,First off I want to say that having lurked on ...
17533,i just drinked a bottle of cologne (I did not ...
36609,Morninggggg how am i just up now when my alarm...


In [341]:
#Data Clean up 
def dataCleanUp(df):
    df = df.str.replace(f"[{string.punctuation}]", "", regex=True)
    df = df.str.lower()
    df = df.apply(lambda x: x if len(x) > 10 else "")# Example: removing short sentences
    return df   


class DataCleanUpTransformer():
    def __init__(self, cleanup_func) -> None:
        self.cleanup_func = cleanup_func
        
    def fit(self, i, y=None):
        return self

    def transform(self, X):
        X_cleaned = self.cleanup_func(X.copy())
        return X_cleaned

# print(y.unique())

In [331]:
categorical_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer())
])


In [ ]:
# Train a logistic regression model
model = LogisticRegression(max_iter=1000)


In [342]:
main_pipeline = Pipeline(steps=[
    ('cleanup', DataCleanUpTransformer(dataCleanUp)),
    ('preprocessing', categorical_transformer),
    ('model', model)
])

main_pipeline.fit(X_train['statement'], y_train)
main_pipeline

Pipeline(steps=[('cleanup',
                 <__main__.DataCleanUpTransformer object at 0x166f5b0d0>),
                ('preprocessing',
                 Pipeline(steps=[('vectorizer', TfidfVectorizer())])),
                ('model', LogisticRegression(max_iter=1000))])

In [343]:
 # Predict the labels for the test set
y_pred = main_pipeline.predict(X_test['statement'])

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Print classification report
print(classification_report(y_test, y_pred))

#Weighted Avgs:
#   Basic, 1000 iterations = 0.77
#   Solver (Sage) = 0.77


Accuracy: 0.7669724770642202
                      precision    recall  f1-score   support

             Anxiety       0.79      0.74      0.76      1157
             Bipolar       0.86      0.68      0.76       817
          Depression       0.70      0.75      0.72      4585
              Normal       0.85      0.96      0.90      4957
Personality disorder       0.81      0.37      0.51       292
              Stress       0.68      0.45      0.54       789
            Suicidal       0.70      0.65      0.67      3208

            accuracy                           0.77     15805
           macro avg       0.77      0.66      0.70     15805
        weighted avg       0.76      0.77      0.76     15805

